In [ ]:
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline

import gala.dynamics as gd
import gala.potential as gp
import gala.integrate as gi
from gala.units import galactic
from gala.dynamics import mockstream

from uncluster.potential import mw_potential

In [ ]:
mw_potential._value(np.array([[1.,2.,3.]]), np.array([0.]))

In [ ]:
mw_potential._gradient(np.array([[1.,2.,3.]]), np.array([0.]))

In [ ]:
static_mw_potential = gp.CCompositePotential()

static_mw_potential['halo'] = gp.NFWPotential(m=mw_potential['halo'].parameters['m0'], 
                                              r_s=mw_potential['halo'].parameters['r_s'], 
                                              units=galactic)

for key in mw_potential.keys():
    if key == 'halo': continue
    
    Class = mw_potential[key].__class__.__name__.lstrip('Growing')
    
    pars = dict()
    for key2 in mw_potential[key].parameters.keys():
        pars[key2.rstrip('0')] = mw_potential[key].parameters[key2]
        
    static_mw_potential[key] = getattr(gp, Class)(**pars, units=galactic)

In [ ]:
derp = np.random.random(size=(3,10))

In [ ]:
%%timeit 
for i in range(100):
    mw_potential._gradient(derp, np.array([0.]))

In [ ]:
%%timeit 
for i in range(100):
    static_mw_potential._gradient(derp, np.array([0.]))

In [ ]:
m_h = pot1.parameters['m0']
r_s = pot1.parameters['r_s']
rr = 15.*u.kpc
m_h * (np.log(1+rr/r_s) - (rr/r_s)/(1 + rr/r_s))

In [ ]:
xyz = [15.,0,0]*u.kpc
pot1.mass_enclosed(xyz), pot2.mass_enclosed(xyz)

In [ ]:
xyz = np.zeros((3,256))
xyz[0] = pot1.parameters['r_s'].to(u.kpc).value
ts = np.linspace(0, -11.6, xyz.shape[1]) * u.Gyr
plt.plot(ts, pot1.mass_enclosed(xyz, t=ts))

In [ ]:
orbit = pot1.integrate_orbit([15.,0,0,0,0.15,0], dt=1., n_steps=11000, t1=-11.*u.Gyr, Integrator=gi.DOPRI853Integrator)
_ = orbit.plot()

In [ ]:
orbit = pot2.integrate_orbit([15.,0,0,0,0.15,0], dt=1., n_steps=11000, t1=-11.*u.Gyr, Integrator=gi.DOPRI853Integrator)
_ = orbit.plot()

In [ ]:
%timeit pot1.integrate_orbit([15.,0,0,0,0.15,0], dt=1., n_steps=11000, t1=-11.*u.Gyr, Integrator=gi.DOPRI853Integrator)
%timeit pot2.integrate_orbit([15.,0,0,0,0.15,0], dt=1., n_steps=11000, t1=-11.*u.Gyr, Integrator=gi.DOPRI853Integrator)

In [ ]:
%%timeit 
mw_potential.integrate_orbit([15.,0,0,0,0.15,0], dt=1., n_steps=11000, 
                             t1=-11.*u.Gyr, Integrator=gi.DOPRI853Integrator)

## Growing potential:

In [ ]:
H = gp.Hamiltonian(mw_potential)
orbit = H.integrate_orbit([35.,0,0,0,0.05,0], dt=1., n_steps=11000, 
                          t1=-11.*u.Gyr, Integrator=gi.DOPRI853Integrator)
_ = orbit.plot()

In [ ]:
%%time
stream = mockstream.fardal_stream(H, orbit, 2E5*u.Msun, release_every=16)

In [ ]:
_ = stream.plot()

## Static potential:

In [ ]:
H2 = gp.Hamiltonian(static_mw_potential)
orbit2 = H2.integrate_orbit([35.,0,0,0,0.05,0], dt=1., n_steps=11000, 
                           t1=-11.*u.Gyr, Integrator=gi.DOPRI853Integrator)
_ = orbit2.plot()

In [ ]:
%%time
stream2 = mockstream.fardal_stream(H2, orbit2, 2E5*u.Msun, release_every=16)

In [ ]:
_ = stream2.plot()

---

In [ ]:
import h5py
from uncluster.utils import quantity_from_hdf5
from uncluster.config import t_max

In [ ]:
w0 = gd.CartesianPhaseSpacePosition(pos=[11.,0,0]*u.kpc, 
                                    vel=[0, 180.,0]*u.km/u.s)

In [ ]:
H1 = gp.Hamiltonian(mw_potential)
orbit1 = H1.integrate_orbit(w0, t1=t_max, t2=0., n_steps=8192, Integrator=gi.DOPRI853Integrator)
_ = orbit1.plot()

In [ ]:
H2 = gp.Hamiltonian(static_mw_potential)
orbit2 = H2.integrate_orbit(w0, t1=t_max, t2=0., n_steps=8192, Integrator=gi.DOPRI853Integrator)
_ = orbit2.plot()

In [ ]:
%%time
stream1 = mockstream.fardal_stream(H1, orbit1, 5E5*u.Msun,
                                   release_every=4,
                                   Integrator=gi.DOPRI853Integrator)

In [ ]:
%%time
stream2 = mockstream.fardal_stream(H2, orbit2, 5E5*u.Msun,
                                   release_every=64,
                                   Integrator=gi.DOPRI853Integrator)

In [ ]:
_ = stream2.plot()